# Preprocessing

In [ ]:
import json
import pandas as pd
import numpy as np
from io import StringIO

import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
import matplotlib.pyplot as plt
from sklearn.base import TransformerMixin


# NLTK Packages
import nltk
#nltk.download( 'vader_lexicon' )
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# Spacy Packages
import spacy
#from spacy.lang.en.stop_words import STOP_WORDS


# Models used for classification
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB

# Feature Extractor for text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#import skipthoughts

In [ ]:
## Import Data
#C:\Users\coope\Data_Science\Datasets\Yelp\Json
#C:\Users\coope\Data_Science\Datasets\Yelp\Json\yelp_academic_dataset_business.json
input_path_business_info = 'C:\\Users\\coope\\Data_Science\\Datasets\\Yelp\\Json\\yelp_academic_dataset_business.json'
input_path_review_data = 'C:\\Users\\coope\\Data_Science\\Datasets\\Yelp\\Json\\yelp_academic_dataset_review.json'
output_path_business_id = 'C:\\Users\\coope\\Data_Science\\Datasets\\Yelp\\business_id.csv'
output_path_total_data = 'C:\\Users\\coope\\Data_Science\\Datasets\\Yelp\\total_data.csv'

In [ ]:
df = pd.read_json(input_path_business_info, lines = True)

df = df.fillna("")

In [ ]:
resturants = df[df["categories"].str.contains("Restaurants")]

In [ ]:
res_id = resturants["business_id"]

In [ ]:
chunks = pd.read_json(path_review, lines = True, chunksize = 10000)
#i = 0
total_data = pd.DataFrame()

for chunk in chunks:
    #if i >= 500:
        #break
    #i += 1
    resturant = chunk[chunk["business_id"].isin(res_id)][["stars", "text"]]
    total_data = pd.concat([total_data, resturant])

In [ ]:
total_data["text"][0]
total_data["stars"][0]

total_data.to_csv(output_path_data)

# Dataset Creation

In [ ]:
path = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\yelp_dataset\\total_data.csv'
out_path_binary = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\Raw Data\\Yelp Data\\binary.csv'
out_path_nominal = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\Raw Data\\Yelp Data\\nominal.csv'

df = pd.read_csv(path, index_col = 0)

In [ ]:
plt.hist(df["stars"])
plt.show()

pd.crosstab(index = df["stars"], columns = "prop") / pd.crosstab(index = df["stars"], columns = "prop").sum()
pd.crosstab(index = df["stars"], columns = "prop")

In [ ]:
# Data set under assumption that 1, 2, and 3 are bad, 4 and 5 are good
df["binary"] = ["Positive" if x >= 4 else "Negative" for x in df["stars"]]
#binary

pd.crosstab(index = df["binary"], columns = "prop") / pd.crosstab(index = df["binary"], columns = "prop").sum()
df.to_csv(out_path_binary, columns = ["text", "binary"], index = False)

In [ ]:
# Data set under assumption that 1 and 2 are bad, 4 and 5 are good, and 3 is Neutral
df["nominal"] = ["Positive" if x >= 4 else "Negative" if x <= 2 else "Neutral" for x in df["stars"]]
pd.crosstab(index = df["nominal"], columns = "prop") / pd.crosstab(index = df["nominal"], columns = "prop").sum()

df.to_csv(out_path_nominal, columns = ["text", "nominal"], index = False)

# Text Classifier

In [ ]:
input_path_nominal = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\Raw Data\\Yelp Data\\nominal.csv'
input_path_binary = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\Raw Data\\Yelp Data\\binary.csv'
path = 'C:\\Users\\coope\\OneDrive\\Desktop\\Side_Projects\\yelp_dataset\\total_data.csv'

df = pd.read_csv(path, index_col = 0)
#df = pd.read_csv(input_path_binary)
#df = pd.read_csv(input_path_nominal)

In [ ]:
df_sample = df.sample(20000, random_state = 12345)

#pd.crosstab(index = df["nominal"], columns = "prop")

X_train, X_test, y_train, y_test = train_test_split( df_sample["text"], df_sample["stars"], test_size = 0.30, random_state = 12345)
#X_train, X_test, y_train, y_test = train_test_split( df_sample["text"], df_sample["nominal"], test_size = 0.30, random_state = 12345)

In [ ]:
# TF-IDF bag of n-grams pipeline and logistic regression
pipeline_tfidf_LOG = Pipeline(steps = [('vectorizer', TfidfVectorizer(ngram_range = (1, 2))),
                        ('classifier', LogisticRegression())])


pipeline_tfidf_LOG.fit(X_train, y_train)
pipeline_tfidf_LOG.score(X_test, y_test) # 62.017 %

In [ ]:
# TF-IDF bag of n-grams pipeline and support vector machine (Linear)
pipeline_tfidf_SVM_L = Pipeline(steps = [('vectorizer', TfidfVectorizer(ngram_range = (1, 2))),
                        ('classifier', svm.LinearSVC())])


pipeline_tfidf_SVM_L.fit(X_train, y_train)
pipeline_tfidf_SVM_L.score(X_test, y_test) # 63.067%

In [ ]:
# TF-IDF bag of n-grams pipeline and Multinomial Naive Bayes
pipeline_tfidf_MNB = Pipeline(steps = [('vectorizer', TfidfVectorizer(ngram_range = (1, 2))),
                        ('classifier', MultinomialNB())])


pipeline_tfidf_MNB.fit(X_train, y_train)
pipeline_tfidf_MNB.score(X_test, y_test) # 42.883%

In [ ]:
# TF-IDF bag of n-grams pipeline and Complement Naive Bayes
pipeline_tfidf_CNB = Pipeline(steps = [('vectorizer', TfidfVectorizer(ngram_range = (1, 2))),
                        ('classifier', ComplementNB())])


pipeline_tfidf_CNB.fit(X_train, y_train)
pipeline_tfidf_CNB.score(X_test, y_test) # 45.467%